### Edgar Moises Hernandez-Gonzalez
#### 25/06/20
#### Clasificacion de calculo mental en EEG con CNN-2D
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Espectrogramas STFT (3D)
##### Resize proporcional de (45 x n_canales) x 27 [1440x27]

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import time
from scipy.signal import spectrogram
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, roc_curve
from keras.models import load_model

Using TensorFlow backend.


In [ ]:
# todos los canales
x = pd.read_csv("/content/drive/My Drive/MentalCalculation/Datos/MeCa_car_01_3.csv", header=None)

# etiquetas
y = pd.read_csv("/content/drive/My Drive/MentalCalculation/Datos/EtiquetasMeCa01ACLL-03.csv", header=None).values

In [ ]:
print(x.shape)
print(y.shape)

(304, 43008)
(304, 1)


In [ ]:
# unir 32 canales y resize de tamaño proporcional
def unir_espectrogramas_vertical(data):
  fs = 512
  datos = np.zeros((data.shape[0],1440,27))
  temporal = np.zeros((1440,27))
  for i in range(data.shape[0]): #n muestras
    for j in range(32): #n canales
      sig = data.iloc[i, j*1344:(j+1)*1344]
      #sig = sig[160:1184]
      #espectrograma de STFFT
      f, t, Sxx = spectrogram(sig, fs=512, window='hann', nperseg=512,
                              noverlap=480, nfft=1024, scaling='spectrum')
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [ ]:
# llamar a unir_espectrogramas_vertical
inicio = time.time()

x = unir_espectrogramas_vertical(x)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
Tiempo: 11.127953052520752


In [ ]:
print(x.shape)

(304, 1440, 27)


In [ ]:
# convertir a float
x = x.astype('float32')

# escalar los valores en un rango de 0 a 1 (normalizar)
x /= np.ceil(np.max(x))

In [ ]:
# convertir de 3D a 4D
x = x.reshape((x.shape[0], x.shape[1], x.shape[2], 1))

print(x.shape)

(304, 1440, 27, 1)


In [ ]:
model = load_model('/content/drive/My Drive/MentalCalculation/Modelos/Model_STFT_01_1_CV.hdf5')

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 1440, 27, 2)       20        
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 720, 13, 2)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 720, 13, 2)        38        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 360, 6, 2)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4320)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                138272    
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)               

In [ ]:
test_loss, test_acc = model.evaluate(x, y, verbose=0)

In [ ]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 1.1823
accuracy: 0.5888


In [ ]:
probabilidades = model.predict(x)
y_pred_todo = np.argmax(probabilidades, 1) #1=fila

In [ ]:
f1 = f1_score(y, y_pred_todo, average='macro')
print("f1:", f1)

f1: 0.37060041407867494


In [ ]:
matriz_confusion = confusion_matrix(y, y_pred_todo)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[  0 125]
 [  0 179]]


In [ ]:
probabilidades = probabilidades[:, 1]
auc = roc_auc_score(y, probabilidades)
#fpr, tpr, thresholds = roc_curve(y, probabilidades)

print("AUC: %.4f" % auc)
#plt.plot(fpr, tpr)
#plt.grid()
#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
#plt.show()

AUC: 0.7696
